# City Zone Creation

## Zone Network Functions

### Setting Up

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic
from shapely import wkt

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

In [3]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [4]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

#### Graph and Features

In [5]:
# NOTE: SELECT THE CITY HERE, COMMENT OUT THE REMAINING CITIES
# select_city = "Manila, Philippines"
# city_file = 'map/Manila.graphml'

# select_city = "Makati, Philippines"
# city_file = 'map/Makati.graphml'

select_city = "Mandaluyong, Philippines"
city_file = 'map/Mandaluyong.graphml'


# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    mode = 'drive'
    graph = ox.graph_from_place(select_city, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, city_file) # Save it as a file

def load_graph():
    graph = ox.load_graphml(city_file)
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph

# NOTE: Only run this if you do not have the graph
generate_graph()

# THIS IS THE MAIN GRAPH FOR THE CITY TO BE USED FOR ALL FUNCTIONS
CITY_GRAPH = load_graph()

Graph loaded successfully
NUMBER OF EDGES:  2374
NUMBER OF NODES:  989




In [6]:
### For Filtering the roads and other features
# GETTING ROADS AND WATERWAYS

# Get all the roads in Manila
road = ox.graph_to_gdfs(CITY_GRAPH,nodes=False, edges=True)


# Get all the roads that are not junctions (ex. Roundabouts, intersection, etc.)
filtered_roads = road[road['junction'].isna()]

# Separate roads whose widths are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']
separation_options = ['primary', 'secondary', 'tertiary', 'unclassified']

# Get the roads whose widths are above the threshold
def check_list(lst):
    return any(x in filter_options for x in lst)

# Download OpenStreetMap data for the area of interest
waterways = ox.features_from_place(select_city, tags={'waterway': True})
filtered_rivers = waterways[waterways['waterway'].isin(['river'])]
filtered_streams = waterways[waterways['waterway'].isin(['stream'])]

# Get all the roads with the allowed road types
filtered_roads_strings = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 
filtered_roads_lists = rows_with_lists[rows_with_lists['highway'].apply(check_list)]

In [7]:
# This function will find which road or river intersects between amenities
# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
filtered_rivers_sindex = filtered_rivers.sindex
filtered_streams_sindex = filtered_streams.sindex

def find_intersecting_features(line):
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(line.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if line.intersects(row['geometry']) and row['highway'] in separation_options:
            return True

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(line.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if line.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in separation_options for x in list_highway):
                return True
    
    # Check intersection with filtered rivers
    possible_matches_rivers = filtered_rivers.iloc[list(filtered_rivers_sindex.intersection(line.bounds))]
    for index, row in possible_matches_rivers.iterrows():
        if line.intersects(row['geometry']):
            return True

    # Check intersection with filtered streams
    possible_matches_streams = filtered_streams.iloc[list(filtered_streams_sindex.intersection(line.bounds))]
    for index, row in possible_matches_streams.iterrows():
        if line.intersects(row['geometry']):
            return True
    
    return False

### Reading Data

In [8]:
# MANILA POPULATION DATA
# Reading population data
manila_population_df = pd.read_csv('manila-population-polygon.csv')
manila_population_df['geometry'] = manila_population_df['geometry'].apply(wkt.loads)
manila_population_gdf = gpd.GeoDataFrame(manila_population_df, crs='epsg:3123')

# Create a base map centered around Manila
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add points to the map
for index, row in manila_population_gdf.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=1,  # Adjust the radius as needed for population density representation
                        color='blue',  # Change color as needed
                        fill=True,
                        fill_color='blue'  # Change fill color as needed
                        ).add_to(m)
    
m.save('Population Maps/population_Manila.html') # Save the map to an HTML file

In [9]:
# MAKATI POPULATION DATA
# Reading population data
makati_population_df = pd.read_csv('makati-population-polygon.csv')
makati_population_df['geometry'] = makati_population_df['geometry'].apply(wkt.loads)
makati_population_gdf = gpd.GeoDataFrame(makati_population_df, crs='epsg:3123')

# Create a base map centered around Manila
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add points to the map
for index, row in makati_population_gdf.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=1,  # Adjust the radius as needed for population density representation
                        color='blue',  # Change color as needed
                        fill=True,
                        fill_color='blue'  # Change fill color as needed
                        ).add_to(m)
    
m.save('Population Maps/population_Makati.html') # Save the map to an HTML file

In [10]:
# MANDALUYONG POPULATION DATA
# Reading population data
mandaluyong_population_df = pd.read_csv('mandaluyong-population-polygon.csv')
mandaluyong_population_df['geometry'] = mandaluyong_population_df['geometry'].apply(wkt.loads)
mandaluyong_population_gdf = gpd.GeoDataFrame(mandaluyong_population_df, crs='epsg:3123')

# Create a base map centered around Manila
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add points to the map
for index, row in mandaluyong_population_gdf.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=1,  # Adjust the radius as needed for population density representation
                        color='blue',  # Change color as needed
                        fill=True,
                        fill_color='blue'  # Change fill color as needed
                        ).add_to(m)
    
m.save('Population Maps/population_Mandaluyong.html') # Save the map to an HTML file

### Initial City Network Creation

In [11]:
# Buckle up. We're trying to create a network out of this monstrosity of a dataframe
# Create a networkx graph

def create_network(amenities_polygon_gdf, amenities_point_gdf):
    amenities_network = nx.Graph()

   # Add polygon nodes
    for index, row in amenities_polygon_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row and 'amenity_points' in row:
            # Generate a unique node identifier for polygons
            node_id = f"polygon_{index}"
            amenities_network.add_node(node_id, polygon_index=index, geometry=row['geometry'], lat=row['geometry'].centroid.y, lon=row['geometry'].centroid.x, amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], amenity_points=row['amenity_points'])
        else:
            print(f"Skipping row {index} in amenities_polygon_gdf due to missing data.")

    # Add point nodes
    for index, row in amenities_point_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row:
            # Generate a unique node identifier for points
            node_id = f"point_{index}"
            
            # This part checks whether the point is a transportation or not 
            if row['amenity'] == 'transportation':
                isTranspo = True
            else:
                isTranspo = False
            amenities_network.add_node(node_id, point_index=index, geometry=row['geometry'], lat=row['y'], lon=row['x'], amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], is_in_polygon=False, isTranspo = isTranspo)
        else:
            print(f"Skipping row {index} in amenities_point_gdf due to missing data.")
            
    return amenities_network

### For Amenity and Zone Connection

In [12]:
# Before Level 1 - ONLY FOR Manila
# To filter out the residential areas that do not exceed the threshold

def filter_out_low_population_density_residentials(graph_to_filter, pop_graph):
    filtered_graph = nx.Graph()
    
    for node_key, node_data in list(graph_to_filter.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon'] and node_data['amenity'] == "residential areas":
            # Different Keyword
            # is_a_zone - Will be added to the filtered graph as it means that the residential area has pop density above the threshold
            if node_key in pop_graph and pop_graph.nodes[node_key]['is_a_zone']:
                filtered_graph.add_node(node_key, **node_data)
                
        else: # If a point or any polygon taht is not residential area, add to the graph
            if node_data['amenity'] != "residential areas":
                filtered_graph.add_node(node_key, **node_data)
    
    # Should return a filtered graph
    return filtered_graph

In [13]:
# LEVEL 1 - COMBINE AMENITIES BY POLYGON
# Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    idx = rtree_index.Index()
    
    # Iterates through each polygon and then enlarges and gets the intersecting ones for easier iteration later
    for node_key, node_data in graph.nodes.items():
        # Ensure that the bounding box coordinates are passed as a tuple
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
            bounds = enlarged_polygon.bounds
            bounds_float = tuple(float(coord) for coord in bounds)
            numeric_key = int(node_key.split('_')[1])
            idx.insert(numeric_key, bounds_float)
    
    #Iterating through each polygon
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            nodes_to_merge = []
            
            #Check first if it is already in a list of polygons to be merged
            for merge_list in list_to_merge:
                if node_key in merge_list:
                    nodes_to_merge = merge_list
                    break
            
            # If this is a new node that is not part of any list, add itself to the list for merging later
            if not nodes_to_merge:
                nodes_to_merge.append(node_key)
            
            # Distance 
            total_distance = 0 # This is to calculate the total distance
            combined_node = graph.nodes[node_key]['geometry']
            
            # Then iterate through other polygons that intersect that polygon based on bounds
            for other_node_key in idx.intersection(node_data['geometry'].bounds):
                formatted_key = f"polygon_{other_node_key}"
                other_node_data = graph.nodes[formatted_key]
                if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                    # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                    if node_key != formatted_key and node_data['amenity'] == other_node_data['amenity']:
                        distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                        if distance <= max_distance:
                            line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                            amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
                            
                            # Check if it does not exceed the max perimeter
                            combined_node = shapely.ops.unary_union([combined_node, graph.nodes[formatted_key]['geometry']])
                            total_distance += degrees_to_meters(combined_node.length)
                            
                            if not amenities_intersecting and total_distance < max_perimeter and not find_intersecting_features(line_between_centroids):
                                nodes_to_merge.append(formatted_key)
            
            if nodes_to_merge not in list_to_merge:
                list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later
                
            
                
    temp_graph = to_graph(list_to_merge)
    lists = graph_to_list(temp_graph)


    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
        combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None

In [14]:
# To merge duplicates in lists
def to_graph(nodes):
    G = nx.Graph()
    for part in nodes:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(nodes):
    it = iter(nodes)
    last = next(it)

    for current in it:
        yield last, current
        last = current
        
def graph_to_list(G):
    connected_components = nx.connected_components(G)
    lists = [list(component) for component in connected_components]
    return lists

In [15]:
# LEVEL 2 - Function to connect zones in a network
def create_zone_network(graph, max_distance):
    connect_graph = nx.Graph()
    network_id = 1
    list_to_connect = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
        bounds = enlarged_polygon.bounds
        bounds_float = tuple(float(coord) for coord in bounds)
        numeric_key = int(node_key.split('_')[1])
        idx.insert(numeric_key, bounds_float)
    
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            connect_nodes = []
            
            #Check first if it is already in a list of polygons to be connected
            for connect_list in list_to_connect:
                if node_key in connect_list:
                    connect_nodes = connect_list
                    break
            
            # If this is a new node that is not part of any list, add itself to the list for merging later
            if not connect_nodes:
                connect_nodes.append(node_key)
                
            # If this is not a residential area that is its own zone
            if node_key not in pop_graph or not pop_graph.nodes[node_key]['is_a_zone']:
                for other_node_key in idx.intersection(node_data['geometry'].bounds):
                    formatted_key = f"polygon_{other_node_key}"
                    other_node_data = graph.nodes[formatted_key]
                    if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                        if node_key != formatted_key:

                            distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                            # Check if they are within distance of each other
                            if distance <= max_distance:
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                if not find_intersecting_features(line_between_centroids):
                                    if formatted_key not in pop_graph or not pop_graph.nodes[formatted_key]['is_a_zone']:
                                        connect_nodes.append(formatted_key)
                                
            if connect_nodes not in list_to_connect:
                list_to_connect.append(connect_nodes) # Add to the list to merge the polygons later
                
    temp_graph = to_graph(list_to_connect)
    lists = graph_to_list(temp_graph)

    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
        network_id += 1
        connect_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points, network_id=network_id)

    return connect_graph

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None

In [16]:

# Function to check population density - To be used for the zone connection
# Uses the combined graph
# Formula: Population Density = Total Population / Total Area

def check_residential_population_density(graph, threshold, population_gdf):
    # Create an R-tree index for efficient spatial querying
    idx = rtree_index.Index()
    
    # Populate the R-tree index with points
    for index, row in population_gdf.iterrows():
        idx.insert(index, (row['longitude'], row['latitude'], row['longitude'], row['latitude']))
    
    pop_graph = nx.Graph()
    
    for node_key, node_data in graph.nodes.items():
        # Check if its a polygon and is a residential area
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon'] and node_data['amenity'] == "residential areas":
            total_pop = 0
            
            # Query the R-tree index to find points within the polygon
            for point_idx in idx.intersection(node_data['geometry'].bounds):
                point = population_gdf.loc[point_idx]
                if Point(point['longitude'], point['latitude']).within(node_data['geometry']):
                    total_pop += point['phl_general_2020']  # Add the density
            
            if total_pop > threshold:
                node_data["is_a_zone"] = True
            else:
                node_data["is_a_zone"] = False
            
            pop_graph.add_node(node_key, density=total_pop, **node_data)
    return pop_graph

#### Visualization

In [17]:
# 1 - Function to plot/visualize main graph network on the map
def plot_network_on_map(amenities_network, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    amenity_colors = {
        'education': 'green',
        'finance': 'blue',
        'government offices': 'red',
        'government areas': 'red',
        'government' : 'red',
        'grocery': 'orange',
        'health': 'magenta',
        'malls': 'yellow',
        'residential areas': 'brown',
        'security': 'gray',
        'transportation': 'lightblue',
        'groceries' : 'indigo',
        'mall' : 'violet',
        'others': 'black'
    }

    # Iterate over the nodes in the network
    for node, data in amenities_network.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                # Plot polygons or multipolygons
                color = amenity_colors[data.get('amenity')]
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [18]:
# This is to visualize the stops
def plot_stops_on_map(network_map, stops, initial_location=[0, 0], zoom_start=10):
    # Iterate over the nodes in the network
    for stop in stops:
        folium.Marker(location=[stop.lat, stop.long], popup=f"{stop.isTranspo}").add_to(network_map)
        
    return network_map

In [19]:
# 2 - Function to plot/visualize connected zones on the map
import random

# This is to better visualize the networks
def plot_connected_zones_network_on_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Cyan", "Magenta", "Maroon",
    "Olive", "Lime", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo", "Violet"]
    
    color_map = {}

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                
                network_id = data["network_id"]
                
                if network_id not in color_map:
                    color = random.choice(colors)
                    color_map[network_id] = color
                else:
                    color = color_map[network_id]
                
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [20]:
# 2.5 - Function to plot/visualize residential areas based on population density on the map
# This is to better visualize which residential areas can become zones
def plot_population_zones_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    
                    if (data['is_a_zone']):
                        folium.Polygon(locations=coordinates, fill=True, color="green", fill_opacity=0.4).add_to(m)
                    else:
                        folium.Polygon(locations=coordinates, fill=True, color="red", fill_opacity=0.4).add_to(m)

    # Return the map
    return m

## -- RUN CITIES BELOW --

## Manila Test

### Reading Data

In [21]:
# READING DATA (All Amenities in Manila)
merged_amenities_points_gdf = gpd.read_file('./City Data/Manila City/Manila_point.geojson')
merged_amenities_polygons_gdf= gpd.read_file('././City Data/Manila City/Manila_polygon.geojson')

merged_amenities_polygons_gdf['amenity_points'] = None

In [22]:
# Getting total area to be used for the area connection type
merged_amenities_polygons_gdf['area'] = degrees_to_meters(merged_amenities_polygons_gdf['geometry'].area)
manila_area_sum = merged_amenities_polygons_gdf['area'].sum()

manila_area_sum

/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_4045/2666581131.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged_amenities_polygons_gdf['area'] = degrees_to_meters(merged_amenities_polygons_gdf['geometry'].area)


135.26544097431915

In [23]:
# Create spatial index for points
idx = rtree_index.Index()
for j, point in merged_amenities_points_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in merged_amenities_polygons_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = merged_amenities_points_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    merged_amenities_polygons_gdf.at[i, 'amenity_points'] = points_within_polygon

### Creating the Zones

In [ ]:
Manila_pikl_filepath = "Saved Networks/Manila/"
Manila_map_filepath = "Saved Maps/Manila/"

In [ ]:
# CREATING INITIAL NETWORK

merged_amenities_network_Manila = create_network(merged_amenities_polygons_gdf, merged_amenities_points_gdf)

# Make a before map
merge_map_Manila = plot_network_on_map(merged_amenities_network_Manila, initial_location=[0, 0], zoom_start=100)
merge_map_Manila.save(f'{Manila_map_filepath}merge_map.html') # Save the map to an HTML file

# EXPORTING THE INITIAL MANILA NETWORK GRAPH
path = f'{Manila_pikl_filepath}Manila_Initial_Network.pkl'
export_networks(merged_amenities_network_Manila, path)

In [ ]:
# LEVEL 0.5 - FILTERING OUT RESIDENTIAL AREAS THAT HAVE LOW POPULATION DENSITY
threshold = 120
filter_residential_graph = check_residential_population_density(merged_amenities_network_Manila, threshold, manila_population_gdf)
filtered_manila_amenities_network = filter_out_low_population_density_residentials(merged_amenities_network_Manila, filter_residential_graph)

# Make an after map for the filtered manila amenities network (Filtered residential areas)
after_residential_filter_map = plot_network_on_map(filtered_manila_amenities_network, initial_location=[0, 0], zoom_start=100)
after_residential_filter_map.save(f'{Manila_map_filepath}residential_filter.html') # Save the map to an HTML file

# EXPORTING THE MANILA POPULATION GRAPH
path = f'{Manila_pikl_filepath}Manila_Population_Graph.pkl'
export_networks(filter_residential_graph, path)

# EXPORTING THE FILTERED MANILA NETWORK GRAPH
path = f'{Manila_pikl_filepath}Manila_Filtered_Network.pkl'
export_networks(filtered_manila_amenities_network, path)

In [ ]:
# LEVEL 1 - CONNECTING POLYGONS OF SAME AMENITY

connected_lines = []
combined_graph_Manila = combine_amenities_by_polygon(filtered_manila_amenities_network, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph_Manila, initial_location=[0, 0], zoom_start=100)


# The lines to show the networks
for line in connected_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(after_map)
after_map.save(f'{Manila_map_filepath}after_map.html') # Save the map to an HTML file

# EXPORTING THE LEVEL 1 GRAPH
path = f'{Manila_pikl_filepath}Manila_Combined_Amenities_Network.pkl'
export_networks(combined_graph_Manila, path)

In [ ]:
pop_graph = check_residential_population_density(combined_graph_Manila, 100, manila_population_gdf)
pop_map = plot_population_zones_map(pop_graph, initial_location=[0, 0], zoom_start=100)

In [ ]:
# LEVEL 2 - CREATING NETWORKS OF AMENITIES

graph_networks_of_polygons_Manila = create_zone_network(graph=combined_graph_Manila, max_distance=100)
networks_map_Manila = plot_connected_zones_network_on_map(graph_networks_of_polygons_Manila, initial_location=[0, 0], zoom_start=100)
networks_map_Manila.save(f'{Manila_map_filepath}networks_map.html') # Save the map to an HTML file

# EXPORTING THE LEVEL 2 GRAPH
path = f'{Manila_pikl_filepath}Manila_Zone_Network.pkl'
export_networks(graph_networks_of_polygons_Manila, path)

## Makati Test

### Reading Data

In [ ]:
# READING DATA (All Amenities in Makati)
Makati_amenities_points_gdf = gpd.read_file('./City Data/Makati City/Makati_point.geojson', crs='epsg:3123')
Makati_amenities_polygons_gdf = gpd.read_file('././City Data/Makati City/Makati_polygon.geojson', crs='epsg:3123')

Makati_amenities_polygons_gdf['amenity_points'] = None

In [ ]:
# Getting total area to be used for the area connection type
Makati_amenities_polygons_gdf['area'] = degrees_to_meters(Makati_amenities_polygons_gdf['geometry'].area)
makati_area_sum = Makati_amenities_polygons_gdf['area'].sum()

makati_area_sum

In [ ]:
# Create spatial index for points
idx = rtree_index.Index()
for j, point in Makati_amenities_points_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in Makati_amenities_polygons_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = Makati_amenities_points_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    Makati_amenities_polygons_gdf.at[i, 'amenity_points'] = points_within_polygon

### Creating the Zones

In [ ]:
Makati_pikl_filepath = "Saved Networks/Makati/"
Makati_map_filepath = "Saved Maps/Makati/"

In [ ]:
# CREATING INITIAL NETWORK

merged_amenities_network_Makati = create_network(Makati_amenities_polygons_gdf, Makati_amenities_points_gdf)

# Make a before map
merge_map_Makati = plot_network_on_map(merged_amenities_network_Makati, initial_location=[0, 0], zoom_start=100)
merge_map_Makati.save(f'{Makati_map_filepath}merge_map.html') # Save the map to an HTML file

# EXPORTING THE INITIAL MAKATI NETWORK GRAPH
path = f'{Makati_pikl_filepath}Makati_Initial_Network.pkl'
export_networks(merged_amenities_network_Makati, path)

In [ ]:
# LEVEL 1 - CONNECTING POLYGONS OF SAME AMENITY

connected_lines = []
combined_graph_Makati = combine_amenities_by_polygon(merged_amenities_network_Makati, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph_Makati, initial_location=[0, 0], zoom_start=100)


# The lines to show the networks
for line in connected_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(after_map)
after_map.save(f'{Makati_map_filepath}after_map.html') # Save the map to an HTML file

# EXPORTING THE LEVEL 1 GRAPH
path = f'{Makati_pikl_filepath}Makati_Combined_Amenities_Network.pkl'
export_networks(combined_graph_Makati, path)

In [ ]:
combined_graph_Makati = import_networks(f'{Makati_pikl_filepath}Makati_Combined_Amenities_Network.pkl')

pop_graph = check_residential_population_density(combined_graph_Makati, 100, makati_population_gdf)
pop_map = plot_population_zones_map(pop_graph, initial_location=[0, 0], zoom_start=100)

# EXPORTING THE MAKATI POPULATION GRAPH
path = f'{Makati_pikl_filepath}Makati_Population_Graph.pkl'
export_networks(pop_graph, path)

In [ ]:
# LEVEL 2 - CREATING NETWORKS OF AMENITIES

graph_networks_of_polygons_Makati= create_zone_network(graph=combined_graph_Makati, max_distance=100)
networks_map_Makati = plot_connected_zones_network_on_map(graph_networks_of_polygons_Makati, initial_location=[0, 0], zoom_start=100)
networks_map_Makati.save(f'{Makati_map_filepath}networks_map.html') # Save the map to an HTML file

# EXPORTING THE LEVEL 2 GRAPH
path = f'{Makati_pikl_filepath}Makati_Zone_Network.pkl'
export_networks(graph_networks_of_polygons_Makati, path)

## Mandaluyong Test

### Reading Data

In [ ]:
# READING DATA (All Amenities in Mandaluyong)
Mandaluyong_amenities_points_gdf = gpd.read_file('./City Data/Mandaluyong City/Mandaluyong_point.geojson', crs='epsg:3123')
Mandaluyong_amenities_polygons_gdf = gpd.read_file('././City Data/Mandaluyong City/Mandaluyong_polygon.geojson', crs='epsg:3123')

Mandaluyong_amenities_polygons_gdf['amenity_points'] = None

In [ ]:
Mandaluyong_amenities_polygons_gdf['area'] = degrees_to_meters(Mandaluyong_amenities_polygons_gdf['geometry'].area)
mandaluyong_area_sum = Mandaluyong_amenities_polygons_gdf['area'].sum()

mandaluyong_area_sum

In [ ]:
# Create spatial index for points
idx = rtree_index.Index()
for j, point in Mandaluyong_amenities_points_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in Mandaluyong_amenities_polygons_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = Mandaluyong_amenities_points_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    Mandaluyong_amenities_polygons_gdf.at[i, 'amenity_points'] = points_within_polygon

### Creating the Zones

In [ ]:
Mandaluyong_pikl_filepath = "Saved Networks/Mandaluyong/"
Mandaluyong_map_filepath = "Saved Maps/Mandaluyong/"

In [ ]:
# CREATING INITIAL NETWORK

merged_amenities_network_Mandaluyong = create_network(Mandaluyong_amenities_polygons_gdf, Mandaluyong_amenities_points_gdf)

# Make a before map
merge_map_Mandaluyong = plot_network_on_map(merged_amenities_network_Mandaluyong, initial_location=[0, 0], zoom_start=100)
merge_map_Mandaluyong.save(f'{Mandaluyong_map_filepath}merge_map.html') # Save the map to an HTML file

# EXPORTING THE INITIAL MANDALUYONG NETWORK GRAPH
path = f'{Mandaluyong_pikl_filepath}Mandaluyong_Initial_Network.pkl'
export_networks(merged_amenities_network_Mandaluyong, path)

In [ ]:
# LEVEL 1 - CONNECTING POLYGONS OF SAME AMENITY

connected_lines = []
combined_graph_Mandaluyong = combine_amenities_by_polygon(merged_amenities_network_Mandaluyong, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph_Mandaluyong, initial_location=[0, 0], zoom_start=100)


# The lines to show the networks
for line in connected_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(after_map)
after_map.save(f'{Mandaluyong_map_filepath}after_map.html') # Save the map to an HTML file

# EXPORTING THE LEVEL 1 GRAPH
path = f'{Mandaluyong_pikl_filepath}Mandaluyong_Combined_Amenities_Network.pkl'
export_networks(combined_graph_Mandaluyong, path)

In [ ]:
combined_graph_Mandaluyong = import_networks(f'{Mandaluyong_pikl_filepath}Mandaluyong_Combined_Amenities_Network.pkl')

pop_graph = check_residential_population_density(combined_graph_Mandaluyong, 100, mandaluyong_population_gdf)
pop_map = plot_population_zones_map(pop_graph, initial_location=[0, 0], zoom_start=100)

# EXPORTING THE MANDALUYONG POPULATION GRAPH
path = f'{Mandaluyong_pikl_filepath}Mandaluyong_Population_Graph.pkl'
export_networks(pop_graph, path)

In [ ]:
# LEVEL 2 - CREATING NETWORKS OF AMENITIES

graph_networks_of_polygons_Mandaluyong= create_zone_network(graph=combined_graph_Mandaluyong, max_distance=100)
networks_map_Mandaluyong = plot_connected_zones_network_on_map(graph_networks_of_polygons_Mandaluyong, initial_location=[0, 0], zoom_start=100)
networks_map_Mandaluyong.save(f'{Mandaluyong_map_filepath}networks_map.html') # Save the map to an HTML file

# EXPORTING THE LEVEL 2 GRAPH
path = f'{Mandaluyong_pikl_filepath}Mandaluyong_Zone_Network.pkl'
export_networks(graph_networks_of_polygons_Mandaluyong, path)

## TEST

In [25]:
# Create a Folium map centered around the city
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add point nodes
for index, row in merged_amenities_points_gdf.iterrows():
    # Check if essential columns exist in the row
    if row['amenity'] == "transportation":
        folium.CircleMarker(location=(row['y'], row['x']),
                        radius=1,
                        color='red',
                        fill=True,
                        fill_color='red').add_to(m)
        
# Save the map to an HTML file
m.save("map_with_transpo.html")